In [214]:
import pandas as pd
import numpy as np


df = pd.read_csv('../Grunddatein/OffersData1MillionRows.csv')
name_for_save_file = '../Grunddatein/Zwischendatein/cleaned_data.csv'

asins2 = pd.read_csv('../Grunddatein/ASINS.csv')
#df = pd.read_csv('Offer21Million.csv')
#name_for_save_file = 'cleaned_data_million.csv'

In [215]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 18 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   id               1000000 non-null  int64  
 1   asin             1000000 non-null  object 
 2   price            1000000 non-null  float64
 3   currency         1000000 non-null  object 
 4   time             1000000 non-null  object 
 5   crawlTime        1000000 non-null  object 
 6   condition        1000000 non-null  object 
 7   sellerName       1000000 non-null  object 
 8   sellerId         1000000 non-null  object 
 9   sellerbewertung  753605 non-null   object 
 10  seller_sterne    745503 non-null   object 
 11  lieferdatum      964682 non-null   object 
 12  lieferpreis      964682 non-null   object 
 13  lieferung_durch  1000000 non-null  object 
 14  ranking          1000000 non-null  int64  
 15  buyBoxWinner     1000000 non-null  bool   
 16  numberOfSellers  10

In [216]:
print("Length of the DataFrame:", df.shape[0])
num_true = (df['buyBoxWinner'] == True).sum()
print(f"The Number of Offers winning the BuyBox is {num_true}")
distinct_asins = df['asin'].unique()
asins_complete = asins2['id'].unique()
print("Number of distinct asins:", len(distinct_asins))
set_distinct_asins = set(distinct_asins)
set_asins_complete = set(asins_complete)

asins_not_in_both = set_distinct_asins.symmetric_difference(set_asins_complete)
print("ASINs not present in both:", asins_not_in_both)

Length of the DataFrame: 1000000
The Number of Offers winning the BuyBox is 93396
Number of distinct asins: 988
ASINs not present in both: {'B00L4XU430', 'B08J2VHWM7', 'B09L6247P8', 'B0BH9FKPWM', 'B09W8GB7KY', 'B0B2RRZLY6', 'B000095Z2Z', 'B0741D7LRD', 'B07GB2GC62', 'B09MY48N93', 'B07H4FRQ4T', 'B00TQU7HCO'}


## Transform Time Column to Datetime Format

In [217]:
# convert the 'time' column to datetime format
df['time'] = pd.to_datetime(df['time'], format="%Y-%m-%d-%H")
df.head()

,id,asin,price,currency,time,crawlTime,condition,sellerName,sellerId,sellerbewertung,seller_sterne,lieferdatum,lieferpreis,lieferung_durch,ranking,buyBoxWinner,numberOfSellers,trigByReactive
0,341683,B09SBXZV9V,141.55,€,2023-03-22 16:00:00,2023-03-22 16:54:19.521639,Neu,belli-shop,AOZ9PW800A1WK,(4211 Bewertungen) 100 % positiv in...,5 von 5,"Samstag, 25. März",GRATIS,Amazon,0,True,13,False
1,341684,B0000C72GD,79.90,€,2023-03-22 16:00:00,2023-03-22 16:54:19.522425,Neu,STILE IMMAGINE DIGITAL HD,A16E8RFMSALSSB,(29 Bewertungen) 97 % positiv über ...,5 von 5,29. - 31. März,"9,90 €",STILE IMMAGINE DIGITAL HD,0,True,2,False
2,341685,B0001GRVJQ,55.31,€,2023-03-22 16:00:00,2023-03-22 16:54:19.523125,Neu,amazon,amazon,NaN,NaN,NaN,NaN,Amazon,0,True,8,False
3,341686,B0002CZU1U,273.28,€,2023-03-22 16:00:00,2023-03-22 16:54:19.523736,Neu,Musikhaus Kirstein GmbH,A2LUZCVBLA57KT,(38466 Bewertungen) 95 % positiv in...,4.5 von 5,27. - 28. März,"2,99 €",Musikhaus Kirstein GmbH,0,True,3,False
4,341687,B0002HOS7M,75.62,€,2023-03-22 16:00:00,2023-03-22 16:54:19.524467,Neu,amazon,amazon,NaN,NaN,"Samstag, 25. März",GRATIS,Amazon,0,True,23,False


## Transforming/Adding Missing Rows/Times
Missing Rows are "Replaced" by the rows for the same asin one hour before
Search For Asins with an Hour Gap

In [218]:
import pandas as pd

# assuming your CSV file is called "offers_data.csv"
#df = pd.read_csv('offers_data.csv', parse_dates=['time'], date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d-%H'))

# create a MultiIndex using columns "asin", "time", and "id"
df.set_index(['asin', 'time', 'id'], inplace=True)

# sort the DataFrame by the MultiIndex
df.sort_index(inplace=True)

# Initialize an empty list called asins_and_gaps
asins_and_gaps = []

def check_one_hour_gap(group):
    group = group.reset_index()  # reset index before indexing with missing_gap Series
    time_diff = group['time'].diff()
    one_hour_gap = pd.Timedelta(hours=1)
    missing_gap = time_diff > one_hour_gap

    if missing_gap.any():
        gap_start_time = group['time'][missing_gap].iloc[0] - one_hour_gap
        missing_hour = gap_start_time.strftime("%Y-%m-%d %H")
        asin = group["asin"].iloc[0]
        #print(f'Missing one-hour gap in ASIN {asin} at {missing_hour}')
        #print("-------------------------")
        #print("\n")

        # Add the ASIN and the missing hour to the asins_and_gaps list
        asins_and_gaps.append((asin, missing_hour))

# apply custom function to each group
df.groupby('asin').apply(check_one_hour_gap)

#print("ASINs and gaps:")
#print(asins_and_gaps)
#asins_and_gaps
distinct_asins = {tup[0] for tup in asins_and_gaps}
print(f"For {len(distinct_asins)} there is at least one time Gap")
print(f"In Total there are {len(asins_and_gaps)} time Gaps")

For 987 there is at least one time Gap
In Total there are 987 time Gaps


### Find Rows from one hour before and insert them into missing_data_df

In [219]:
import pandas as pd

# Assuming you already have the DataFrame df and the asins_and_gaps list

# Initialize an empty DataFrame called missing_data_df
missing_data_df = pd.DataFrame()

for asin, gap_time_str in asins_and_gaps:
    # Convert the gap time string to a datetime object
    gap_time = pd.to_datetime(gap_time_str, format='%Y-%m-%d %H')

    matching_rows = pd.DataFrame()
    gap_found = 0

    # Search for rows up to 3 hours earlier
    for hours in range(1, 4):
        time_earlier = gap_time - pd.Timedelta(hours=hours)
        matching_rows = df.loc[(df.index.get_level_values('asin') == asin) & (df.index.get_level_values('time') == time_earlier)]

        if not matching_rows.empty:
            gap_found = -hours
            break

    # If matching rows are found
    if not matching_rows.empty:
        # Make a copy of matching_rows before modifying it
        matching_rows = matching_rows.copy()

        # Update the time column value for the copied rows to the appropriate gap
        matching_rows.reset_index(inplace=True)
        matching_rows[matching_rows.columns[matching_rows.columns.get_loc('time')]] = matching_rows['time'] - pd.Timedelta(hours=gap_found)

        # Add a new column named "time_gap" to indicate the gap at which the rows were found
        matching_rows['time_gap'] = gap_found

        # Append the matching rows to the missing_data_df DataFrame
        missing_data_df = pd.concat([missing_data_df, matching_rows])

# Reset the index of the missing_data_df DataFrame
missing_data_df.reset_index(drop=True, inplace=True)
#missing_data_df.info()
distinct_values = missing_data_df['asin'].nunique()
distinct_values
print(f"For {distinct_values} there could be found rows from 1/2/3 hourls earlier to replace the time gap")

For 976 there could be found rows from 1/2/3 hourls earlier to replace the time gap


## Concat Missing Dataframe to Normal Dataframe

In [220]:
df['time_gap'] = 0
df['copyed_cause_missing'] = False
df = df.reset_index()
missing_data_df['copyed_cause_missing'] = True
old_size = df.shape[0]
df = pd.concat([df, missing_data_df])
new_size = df.shape[0]
del missing_data_df
print(f"The Old Size was {old_size} and the New Size is {new_size}. This Means {new_size-old_size} have been added")

The Old Size was 1000000 and the New Size is 1009221. This Means 9221 have been added


## Save File

In [221]:
## Save Offers_data to File with filled Data
df.to_csv('../Grunddatein/Zwischendatein/offers_without_gaps.csv', index=False)
df = pd.read_csv('../Grunddatein/Zwischendatein/offers_without_gaps.csv')

## Transforming Dataframe Seller Sterne

In [222]:
null_sterne_df = df[df['seller_sterne'].isnull()]
unique_seller_names = null_sterne_df['sellerName'].unique()
unique_seller_names

# get the seller names where 'seller_sterne' is null
null_seller_names = df.loc[df['seller_sterne'].isnull(), 'sellerName']

# check if there are any rows where 'seller_sterne' is not null for the null seller names
has_sterne_values = df.loc[df['sellerName'].isin(null_seller_names) & ~df['seller_sterne'].isnull()]

# print the result
if has_sterne_values.empty:
    print("There are no rows where 'seller_sterne' is not null for the null seller names.")
else:
    print("There are rows where 'seller_sterne' is not null for the null seller names.")

There are rows where 'seller_sterne' is not null for the null seller names.


In [223]:

# define a lambda function to convert the string to a float or return None for null values
str_to_float_or_none = lambda x: float(x.split()[0].replace(",", ".")) if isinstance(x, str) and len(x.split()) > 0 else None

# apply the lambda function to the 'seller_sterne' column to replace the string values with their corresponding float values
df['seller_sterne'] = df['seller_sterne'].apply(str_to_float_or_none)

# select the rows where 'sellerName' does not contain 'amazon'
no_amazon_df = df[~df['sellerName'].str.contains('amazon', case=False)]

# calculate the rounded value of 'seller_sterne'
average_sterne = no_amazon_df['seller_sterne'].mean()
rounded_sterne = round(average_sterne * 2) / 2

# replace the null values in 'seller_sterne' with the rounded value for non-amazon rows, and with 5.0 for amazon rows
df.loc[~df['sellerName'].str.contains('amazon', case=False), 'seller_sterne'] = rounded_sterne
df.loc[df['sellerName'].str.contains('amazon', case=False), 'seller_sterne'] = 5.0

## Transformin Dataframe Lieferpreis


In [224]:
df['lieferpreis'] = df['lieferpreis'].replace(['GRATIS', 'FREE'], 0.0)
##Replace Euro Signs
df['lieferpreis'] = df['lieferpreis'].replace('€', '', regex=True)
df = df.replace(',', '.', regex=True)
df['lieferpreis'] = df['lieferpreis'].astype(float, errors='ignore')
num_missing = df['lieferpreis'].isna().sum()
print(f"The column 'lieferpreis' has {num_missing} missing values.")


The column 'lieferpreis' has 35638 missing values.


In [225]:
null_sterne_df = df[df['seller_sterne'].isnull()]
unique_seller_names = null_sterne_df['sellerName'].unique()
unique_seller_names

array([], dtype=object)

## Transform Dataframe Sellerbewertung


In [226]:
# Split values into two columns based on first space
df[['vorderer_Teil', 'hinterer_Teil']] = df['sellerbewertung'].str.split(')', 1, expand=True)
df['hinterer_Teil'] = df['hinterer_Teil'].str.replace('%.*', '', regex=True)
# Replace NaN values in vorderer_Teil column with 0
df['vorderer_Teil'] = df['vorderer_Teil'].fillna(0)
df['vorderer_Teil'] = df['vorderer_Teil'].str.extract('(\d+)').astype(float)
df['hinterer_Teil'] = df['hinterer_Teil'].astype(float) / 100.0
df['sellerbewertung_ausgerechnet'] = df['vorderer_Teil'] * df['hinterer_Teil']
df['sellerbewertung'] = df['sellerbewertung_ausgerechnet']

# Drop columns that were created in the process
df = df.drop(['vorderer_Teil', 'hinterer_Teil', 'sellerbewertung_ausgerechnet'], axis=1)

C:\Users\Robert Betschinger\AppData\Local\Temp\ipykernel_12788\2410512019.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['vorderer_Teil', 'hinterer_Teil']] = df['sellerbewertung'].str.split(')', 1, expand=True)


In [227]:
#print(df[['sellerName', 'lieferung_durch']].head(5000).to_string(index=False))
df['Fulfillment_type'] = ['FBA' if 'amazon' in x.lower() else 'FBM' for x in df['lieferung_durch']]

## Transform Dataframe Lieferdauer

In [228]:
df['month'] = -1

# loop over each row in the dataframe
for i, row in df.iterrows():
    # check if the 'lieferdatum' value contains the word "April"
    if 'April' in str(row['lieferdatum']):
        # if it does, set the 'month' value to 4
        df.at[i, 'month'] = 4
    # if it does not, check if the 'lieferdatum' value contains the word "März"
    elif 'März' in str(row['lieferdatum']):
        # if it does, set the 'month' value to 3
        df.at[i, 'month'] = 3

In [229]:
import re
df['days'] = [[] for _ in range(len(df.index))]

# loop over each row in the dataframe
for i, row in df.iterrows():
    # extract all the numeric values from the 'lieferdatum' value using regular expressions
    nums = re.findall(r'\d+', str(row['lieferdatum']))
    # add the numeric values as a list to the 'days' column for this row
    df.at[i, 'days'] = [int(num) for num in nums]

In [230]:
# calculate the average value of each non-empty array in the 'days' column
avg_days = df['days'].apply(lambda x: sum(x)/len(x) if x else np.nan)

# replace the 'days' column with the average values
df['days'] = avg_days.astype(float)
#df['lieferdatum']

In [231]:
df['datetime'] = pd.to_datetime(df['time'].str[:10], format='%Y-%m-%d')
# create a new column called 'lieferdatum_datetime' and initialize it to NaN
df['lieferdatum_datetime'] = np.nan

# loop over each row in the dataframe
for i, row in df.iterrows():
    # get the 'month' and 'days' values for this row
    month = row['month']
    days = row['days']
    
    # skip this row if the 'month' or 'days' value is NaN or -1
    if np.isnan(days) or month == -1:
        continue
    
    # create a datetime object for this row and extract only the date portion
    date_str = f"2023-{int(month):02d}-{int(days):02d}"
    datetime_obj = pd.to_datetime(date_str, format='%Y-%m-%d')
    date_only = datetime_obj.date()
    
    # save the date object to the 'lieferdatum_datetime' column for this row
    df.at[i, 'lieferdatum_datetime'] = date_only
df.head()

# convert 'lieferdatum_datetime' column to a Pandas Timestamp object
df['lieferdatum_datetime'] = pd.to_datetime(df['lieferdatum_datetime'])
df['date_diff'] = np.where(df['lieferdatum_datetime'].isna(), np.nan, (df['lieferdatum_datetime'] - df['datetime']).dt.days)
df = df.drop(['days', 'month', 'datetime','lieferdatum_datetime'], axis=1)
df.head()


,asin,time,id,price,currency,crawlTime,condition,sellerName,sellerId,sellerbewertung,...,lieferpreis,lieferung_durch,ranking,buyBoxWinner,numberOfSellers,trigByReactive,time_gap,copyed_cause_missing,Fulfillment_type,date_diff
0,B00000JD6K,2023-03-22 16:00:00,343568,95.8,€,2023-03-22 16:55:20.125842,Neu,Kidia,A2XUKJNGI8V9XU,46.56,...,0.0,Amazon,0,True,2,False,0,False,FBA,3.0
1,B00000JD6K,2023-03-22 16:00:00,344038,86.9,€,2023-03-22 16:55:20.315580,Neu,STILE IMMAGINE DIGITAL HD,A16E8RFMSALSSB,28.13,...,9.9,STILE IMMAGINE DIGITAL HD,1,False,1,False,0,False,FBM,27.0
2,B00000JD6K,2023-03-22 18:00:00,352772,95.8,€,2023-03-22 18:00:22.914723,Neu,Kidia,A2XUKJNGI8V9XU,46.56,...,0.0,Amazon,0,True,2,False,0,False,FBA,3.0
3,B00000JD6K,2023-03-22 18:00:00,353250,86.9,€,2023-03-22 18:00:23.106664,Neu,STILE IMMAGINE DIGITAL HD,A16E8RFMSALSSB,28.13,...,9.9,STILE IMMAGINE DIGITAL HD,1,False,1,False,0,False,FBM,27.0
4,B00000JD6K,2023-03-22 19:00:00,361969,86.9,€,2023-03-22 19:05:33.089028,Neu,STILE IMMAGINE DIGITAL HD,A16E8RFMSALSSB,28.13,...,9.9,STILE IMMAGINE DIGITAL HD,0,True,2,False,0,False,FBM,25.0


## Transform Price

In [232]:
# Select rows where the 'price' column is between 1 and 2
selected_rows = df[(df['price'] >= 1) & (df['price'] <= 2)]
selected_rows.head()

,asin,time,id,price,currency,crawlTime,condition,sellerName,sellerId,sellerbewertung,...,lieferpreis,lieferung_durch,ranking,buyBoxWinner,numberOfSellers,trigByReactive,time_gap,copyed_cause_missing,Fulfillment_type,date_diff
263872,B01FE7K184,2023-03-22 16:00:00,342473,1.118,€,2023-03-22 16:54:19.836242,Neu,GetMarket,AKAKN0OST1C61,59.22,...,9.99,GetMarket,1,False,3,False,0,False,FBM,24.0
263873,B01FE7K184,2023-03-22 16:00:00,342474,1.242,€,2023-03-22 16:54:19.836543,Neu,Solution 4YOU,A3UY995ZKNNQ2M,84.46,...,0.00,Solution 4YOU,2,False,3,False,0,False,FBM,18.0
263874,B01FE7K184,2023-03-22 16:00:00,342475,1.326,€,2023-03-22 16:54:19.836840,Neu,E-Mistero,A1T7OOZ029MPBM,48.40,...,78.57,E-Mistero,3,False,3,False,0,False,FBM,17.0
263875,B01FE7K184,2023-03-22 17:00:00,351643,1.109,€,2023-03-22 17:59:25.866981,Neu,GetMarket,AKAKN0OST1C61,59.22,...,9.99,GetMarket,1,False,3,False,0,False,FBM,7.0
263876,B01FE7K184,2023-03-22 17:00:00,351644,1.242,€,2023-03-22 17:59:25.867302,Neu,Solution 4YOU,A3UY995ZKNNQ2M,84.46,...,0.00,Solution 4YOU,2,False,3,False,0,False,FBM,12.0


In [233]:
#Prices over 1000 € are misinterpreted due to a wrongly placed . sign. Transforming it.
## All Prices below 2 Euros where checked if Products where existing, for which this price is real. All Products under question are actully priced over 1000€

# Select rows where the 'price' column is between 1 and 2
selected_rows = df[(df['price'] >= 1) & (df['price'] <= 2)]

# Multiply the price by 1000 for rows where the price is between 1 and 2
selected_rows.loc[(selected_rows['price'] >= 1) & (selected_rows['price'] <= 2), 'price'] *= 1000

# Update the original DataFrame with the updated values
df.update(selected_rows)

# Print the result
#selected_rows.head()

C:\Users\Robert Betschinger\AppData\Local\Temp\ipykernel_12788\3143074723.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows.loc[(selected_rows['price'] >= 1) & (selected_rows['price'] <= 2), 'price'] *= 1000


In [234]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009221 entries, 0 to 1009220
Data columns (total 22 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   asin                  1009221 non-null  object 
 1   time                  1009221 non-null  object 
 2   id                    1009221 non-null  float64
 3   price                 1009221 non-null  float64
 4   currency              1009221 non-null  object 
 5   crawlTime             1009221 non-null  object 
 6   condition             1009221 non-null  object 
 7   sellerName            1009221 non-null  object 
 8   sellerId              1009221 non-null  object 
 9   sellerbewertung       752384 non-null   float64
 10  seller_sterne         1009221 non-null  float64
 11  lieferdatum           973583 non-null   object 
 12  lieferpreis           973583 non-null   float64
 13  lieferung_durch       1009221 non-null  object 
 14  ranking               1009221 non-

In [236]:
name_for_save_file = '../Grunddatein/Zwischendatein/OffersData1MillionCleaned.csv'

## Save Data to File

In [237]:
df.to_csv(name_for_save_file, index=False)
#Relevant für RF: asin, price,(time),sellerid, sellerbewertung, seller_sterne, lieferpreis, Fulfillment_type, date_diff, ranking, numberOfSellers